In [1]:
"""
Created on Tue Dec 19 07:16:52 2017

@author: Verena Yacoub
"""

import numpy as np
import cv2

cap = cv2.VideoCapture('solidWhiteRight.mp4')
cv2.namedWindow('frames')
while(cap.isOpened()):
    ret, image = cap.read()

    if ret is True:
        gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY) #convert to gray scale
        
             
        kernel_size = 5 #define kernel size for gaussian smoothing
        blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0) #performing smoothing or noise reduction
             
        
        low_threshold = 100 # minimum value below which canny filter classify a signal as non edge
        high_threshold = 200 # maximum value above which a signal is a sure edge
        edges = cv2.Canny(blur_gray, low_threshold, high_threshold) #apply canny edge detection
             
             # Next we'll create a masked edges image using cv2.fillPoly()
        mask = np.zeros_like(edges)   
        ignore_mask_color = 255   
        
        # This time we are defining a four sided polygon to mask
        imshape = image.shape
        vertices = np.array([[(0,imshape[0]),(450, 325), (525,325), (890,imshape[0])]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)     
        
            #returning the image only where mask pixels are nonzero
        masked_edges = cv2.bitwise_and(edges, mask)
              
        
                 # Define the Hough transform parameters
        rho=2           # distance resolution in pixels of the Hough grid
        theta = np.pi/180   # angular resolution in radians of the Hough grid
        threshold= 15       # minimum number of votes (intersections in Hough grid cell)
        min_line_length= 20       # minimum number of pixels making up a line
        max_line_gap= 20       # maximum gap in pixels between connectable line segments
        
        img= np.copy(image)*0      
        lines= cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap)
        # Iterate over the output "lines" and draw lines on a blank image
        #for line in lines:
           # for x1,y1,x2,y2 in line:
                #cv2.line(img,(x1,y1),(x2,y2),(255,255,0),10)
        thickness=15
        color=[0,0,255]       
        # reshape lines to a 2d matrix
        lines = lines.reshape(lines.shape[0], lines.shape[2])
            # create array of slopes
        slopes = (lines[:,3] - lines[:,1]) /(lines[:,2] - lines[:,0])
            # remove junk from lists
        lines = lines[~np.isnan(lines) & ~np.isinf(lines)]
        slopes = slopes[~np.isnan(slopes) & ~np.isinf(slopes)]
            # convert lines into list of points
        lines.shape = (lines.shape[0]//2,2)
        
            # Right lane
            # move all points with negative slopes into right "lane"
            #right_slopes = slopes[slopes < 0]
        right_lines = np.array(list(filter(lambda x: x[0] > (img.shape[1]/2), lines)))
        print(right_lines)
        max_right_x, max_right_y = right_lines.max(axis=0)
        min_right_x, min_right_y = right_lines.min(axis=0)
        
            # Left lane
            # all positive  slopes go into left "lane"
            #left_slopes = slopes[slopes > 0]
        left_lines = np.array(list(filter(lambda x: x[0] < (img.shape[1]/2), lines)))
        max_left_x, max_left_y = left_lines.max(axis=0)
        min_left_x, min_left_y = left_lines.min(axis=0)
        
            # Curve fitting approach
            # calculate polynomial fit for the points in right lane
        right_curve = np.poly1d(np.polyfit(right_lines[:,1], right_lines[:,0], 2))
        left_curve  = np.poly1d(np.polyfit(left_lines[:,1], left_lines[:,0], 2))
        
            # shared ceiling on the horizon for both lines
        min_y = min(min_left_y, min_right_y)
        
            # use new curve function f(y) to calculate x values
        max_right_x = int(right_curve(img.shape[0]))
        min_right_x = int(right_curve(min_right_y))
        
        min_left_x = int(left_curve(img.shape[0]))
        
        r1 = (min_right_x, min_y)
        r2 = (max_right_x, img.shape[0])
        print('Right points r1 and r2,', r1, r2)
        cv2.line(img, r1, r2, color, thickness)
        
        l1 = (max_left_x, min_y)
        l2 = (min_left_x, img.shape[0])
        print('Left points l1 and l2,', l1, l2)
        cv2.line(img, l1, l2, color, thickness)
                
        lines_edges = cv2.addWeighted(image, 0.8, img, 1, 0)
        cv2.imshow('frames', lines_edges)
        k= cv2.waitKey(5) & 0xFF
    
        if k==15:
         break

cv2.destroyAllWindows()
cap.release()  




